In [45]:
import librosa
import pandas as pd
import numpy as np
import 

def extract_audio_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    
    features = {
        
         "pitch": np.mean(librosa.feature.pitch(y=y, sr=sr)),
        "tempo": np.mean(librosa.beat.tempo(y=y, sr=sr)),
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)),
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1),
        "spectral_centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "rms": np.mean(librosa.feature.rms(y=y)),
    }
    
    return features


# Specify the directory where your audio files are stored
audio_directory = r"C:\Users\mohan\Downloads\archive (2)\wav"

# List all .wav files in the directory
audio_files = [os.path.join(audio_directory, file) for file in os.listdir(audio_directory) if file.endswith('.wav')]

# Now you have all .wav file paths stored in the audio_files list
# Optionally, print how many files were found
print(f"Found {len(audio_files)} audio files.")

# Proceed with processing the files (without printing)
file_paths = []  # Initialize a list to store file paths

for audio_file in audio_files:
    file_paths.append(audio_file)
    print( file_paths)

# Assuming you have a list of audio file paths and labels
audio_files = ["audio1.wav", "audio2.wav", ...]
labels = ["Anxiety", "Neutral", ...]  # These labels should be based on your annotations

data = []

for i, audio_file in enumerate(audio_files):
    features = extract_audio_features(audio_file)
    features['label'] = labels[i]
    data.append(features)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.to_csv("annotated_audio_features.csv", index=False)
  

Found 535 audio files.


NameError: name 'file' is not defined

In [50]:
import pandas as pd
import os

# Load the annotated CSV file
csv_file_path = r"C:\Users\mohan\annotated_audio_features.csv"'  # Update this path to your CSV file
df = pd.read_csv(csv_file_path)

# Display the first few rows to understand its structure
print(df.head())

SyntaxError: unterminated string literal (detected at line 5) (16891613.py, line 5)

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load the CSV file containing the audio features
data = pd.read_csv("annotated_audio_features.csv")

# Step 2: Split features and labels
X = data.drop('label', axis=1).values  # Features as NumPy array
y = data['label'].values                # Target labels

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Step 4: Train the SVM Model
# You can experiment with different kernels like 'linear', 'rbf', 'poly', etc.
model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Adjust C and gamma as needed
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


ValueError: could not convert string to float: '[-2.3265572e+02  7.8358467e+01 -5.6844788e+00  2.3088654e+01\n -5.6207581e+00  2.2314155e+00 -1.8465944e+01  1.8867363e-01\n -8.6601562e+00  5.2006960e+00 -8.8264589e+00  1.2440559e+00\n -5.2814016e+00]'

In [38]:
import pandas as pd
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the annotated CSV file
csv_file_path = 'path/to/your/annotations.csv'  # Update this path
df = pd.read_csv(csv_file_path)

# Feature extraction function
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# Initialize lists to store features and labels
features = []
labels = []

# Loop through the dataframe and extract features
for index, row in df.iterrows():
    audio_file_path = os.path.join("path/to/audio/files", row['File Name'])  # Update this path
    mfccs = extract_features(audio_file_path)
    features.append(mfccs)
    labels.append(row['Label'])

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Build a simple neural network model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer for the number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predict on test data
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Decode predicted labels
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print("Predicted Labels:", predicted_labels)


ModuleNotFoundError: No module named 'tensorflow'

In [46]:
import pandas as pd

# Load the CSV file containing the audio features
data = pd.read_csv("annotated_audio_features.csv")

# Display the first few rows of the DataFrame
print(data.head())


        pitch       tempo  zero_crossing_rate  \
0  237.889019  117.187500            0.125008   
1  173.860735  110.294118            0.087326   
2  273.512476  117.187500            0.158534   
3  335.901156  133.928571            0.133681   
4  165.278364  170.454545            0.085460   

                                                mfcc  spectral_centroid  \
0  [-2.3265572e+02  7.8358467e+01 -5.6844788e+00 ...        1845.686773   
1  [-217.2759     111.63815    -12.414565    33.0...        1392.353326   
2  [-178.62926      51.50612     -41.428158     1...        2154.851813   
3  [-210.08412     54.72459     -7.3475137   10.0...        2075.851654   
4  [-1.96598495e+02  1.02991898e+02 -2.64047086e-...        1467.793386   

        rms      label  
0  0.067497      anger  
1  0.102410    boredom  
2  0.106321    anxiety  
3  0.104309  happiness  
4  0.133435    sadness  


In [23]:
from sklearn.preprocessing import LabelEncoder

# Assuming the label column is named 'label'
X = data.drop('label', axis=1)  # Features
y = data['label']                # Target labels

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print ("C")


C


In [24]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print("S")

S


In [52]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Sample data
X_train = np.random.rand(100,10)  # 100 samples, 10 features
y_train = np.random.randint(0, 2, size=(100,))  # 100 labels (0 or 1)

# Check the shape
print("y_train shape:", y_train.shape)

# Reshape if necessary
if len(y_train.shape) > 1:
    y_train = y_train.ravel()

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
joblib.dump(model, 'random_forest_model.joblib')


y_train shape: (100,)


['random_forest_model.joblib']

In [34]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Sample dataset
X = np.random.rand(100, 10)  # 100 samples, 10 features
y = np.random.randint(0, 2, size=(100,))  # Binary labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.55


In [31]:
import librosa
import numpy as np
import joblib
import librosa
import numpy as np
import pandas as pd

# Step 1: Load the trained model
model = joblib.load('random_forest_model.joblib')

def extract_audio_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)
    
    # Compute pitch
    f0, voiced_flag, _ = librosa.core.piptrack(y=y, sr=sr)
    pitch = np.nanmean(f0[voiced_flag > 0])  # Mean pitch, ignoring NaN values

    # Compute tempo using librosa.beat.tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

    # Create features dictionary
    features = {
        "pitch": pitch,
        "tempo": np.mean(tempo),  # Average tempo
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)),
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1),
        "spectral_centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "rms": np.mean(librosa.feature.rms(y=y)),
    }

    return #np.array(list(features.values()))  # Return as a NumPy array
new_audio_features = extract_audio_features(r"C:\Users\mohan\Downloads\Speaker26_000.wav\Speaker26_000.wav")

# Step 4: Check and reshape the features for the model
##print("Type of new_audio_features:", type(new_audio_features))  # Should print <class 'numpy.ndarray'>
#print("Shape of new_audio_features:", new_audio_features.reshape)

# Reshape if necessary
if new_audio_features.ndim == 1:
    new_audio_features = new_audio_features.reshape(1, -1)  # Ensure it's a 2D array

# Step 5: Make a prediction
condition_prediction = model.predict(new_audio_features)
print(f"Predicted Condition: {condition_prediction[0]}")


ValueError: not enough values to unpack (expected 3, got 2)

In [21]:
import librosa
import numpy as np

def extract_audio_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute pitch using librosa.core.piptrack()
    f0, voiced_flag = librosa.core.piptrack(y=y, sr=sr)

    # Calculate mean pitch, ignoring NaN values
    pitch = np.nanmean(f0[voiced_flag > 0])

    # Compute tempo using librosa.beat.tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

    # Create features dictionary
    features = {
        "pitch": pitch,
        "tempo": np.mean(tempo),  # Average tempo
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)),
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1),
        "spectral_centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "rms": np.mean(librosa.feature.rms(y=y)),
    }

    # Return features as a NumPy array
    return np.array(list(features.values()))  # Return as a NumPy array

# Step 4: Use the function
new_audio_features = extract_audio_features(r"C:\Users\mohan\Downloads\Speaker26_000.wav\Speaker26_000.wav")

# Step 5: Check the shape and reshape if necessary
if new_audio_features.ndim == 1:
    # Expand dimensions to ensure it's 2D
    new_audio_features = np.expand_dims(new_audio_features, axis=0)  # Convert to shape (1, n_features)

# Print the shape for verification
print("Shape of new_audio_features:", new_audio_features.shape)


C:\Users\mohan\AppData\Local\Temp\ipykernel_27588\1606484206.py:16: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [23]:
import librosa
import numpy as np

def extract_audio_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute pitch using librosa.core.piptrack()
    f0, voiced_flag = librosa.core.piptrack(y=y, sr=sr)

    # Calculate mean pitch, ignoring NaN values
    pitch = np.nanmean(f0[voiced_flag > 0])

    # Compute tempo using librosa.beat.tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

    # Create features dictionary
    features = {
        "pitch": pitch,
        "tempo": np.mean(tempo),  # Average tempo
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)),
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1).tolist(),  # Ensure it's a list
        "spectral_centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "rms": np.mean(librosa.feature.rms(y=y)),
    }

    # Flatten the features if necessary
    flattened_features = []
    for key in features:
        if isinstance(features[key], list):
            flattened_features.extend(features[key])  # Extend if it's a list
        else:
            flattened_features.append(features[key])  # Append if it's a single value

    # Return features as a NumPy array
    return np.array(flattened_features)  # Return as a NumPy array

# Step 4: Use the function
new_audio_features = extract_audio_features(r"C:\Users\mohan\Downloads\Speaker26_000.wav\Speaker26_000.wav")

# Step 5: Check the shape and reshape if necessary
if new_audio_features.ndim == 1:
    # Expand dimensions to ensure it's 2D
    new_audio_features = np.expand_dims(new_audio_features, axis=0)  # Convert to shape (1, n_features)

# Print the shape for verification
print("Shape of new_audio_features:", new_audio_features.shape)
condition_prediction = model.predict(new_audio_features)
print(f"Predicted Condition: {condition_prediction[0]}")


C:\Users\mohan\AppData\Local\Temp\ipykernel_27588\3413180326.py:16: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)


Shape of new_audio_features: (1, 18)


ValueError: X has 18 features, but RandomForestClassifier is expecting 10 features as input.

In [36]:
import librosa
import numpy as np

def extract_audio_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute pitch using librosa.core.piptrack()
    f0, voiced_flag = librosa.core.piptrack(y=y, sr=sr)

    # Calculate mean pitch, ignoring NaN values
    pitch = np.nanmean(f0[voiced_flag > 0])

    # Compute tempo using librosa.beat.tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

    # Create features dictionary
    features = {
        "pitch": pitch,
        "tempo": np.mean(tempo),  # Average tempo
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)),
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=3), axis=1).tolist(),  # Use only the first 3 MFCCs
        "spectral_centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "rms": np.mean(librosa.feature.rms(y=y)),
    }

    # Flatten the features if necessary
    flattened_features = []
    for key in features:
        if isinstance(features[key], list):
            flattened_features.extend(features[key])  # Extend if it's a list
        else:
            flattened_features.append(features[key])  # Append if it's a single value

    # Return features as a NumPy array
    return np.array(flattened_features)  # Return as a NumPy array

# Step 4: Use the function
new_audio_features = extract_audio_features(r"C:\Users\mohan\Downloads\archive (2)\wav\16b09Lb.wav")

# Step 5: Check the shape and reshape if necessary
if new_audio_features.ndim == 1:
    # Expand dimensions to ensure it's 2D
    new_audio_features = np.expand_dims(new_audio_features, axis=0)  # Convert to shape (1, n_features)

# Print the shape for verification
print("Shape of new_audio_features:", new_audio_features.shape)

# Prediction
condition_prediction = model.predict(new_audio_features)
print(f"Predicted Condition: {condition_prediction[0]}")


Shape of new_audio_features: (1, 8)


C:\Users\mohan\AppData\Local\Temp\ipykernel_27588\1865247655.py:16: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)


ValueError: X has 8 features, but RandomForestClassifier is expecting 10 features as input.

In [37]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Step 1: Define the feature extraction function
def extract_audio_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)

    # Compute features
    pitch = np.nanmean(librosa.core.piptrack(y=y, sr=sr)[0])
    tempo = np.mean(librosa.beat.tempo(y=y, sr=sr))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=8), axis=1)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    rms = np.mean(librosa.feature.rms(y=y))

    # Return the features as a numpy array
    features = np.array([
        pitch,
        tempo,
        zero_crossing_rate,
        *mfccs,  # Unpacking MFCCs (8 values)
        spectral_centroid,
        spectral_bandwidth,
        chroma_stft,
        rms
    ])

    return features

# Step 2: Load your dataset and extract features
# Assuming you have a CSV file with audio file paths and labels
data = pd.read_csv('your_dataset.csv')  # Update with your CSV file path

features_list = []
labels = []

for index, row in data.iterrows():
    audio_file = row['audio_file']  # Column name containing file paths
    label = row['label']  # Column name containing labels

    # Extract features
    features = extract_audio_features(audio_file)
    features_list.append(features)
    labels.append(label)

# Convert to numpy arrays
X = np.array(features_list)
y = np.array(labels)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 6: Save the model (optional)
import joblib
joblib.dump(model, 'random_forest_model.joblib')


FileNotFoundError: [Errno 2] No such file or directory: 'your_dataset.csv'

In [44]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Step 1: Define the feature extraction function
def extract_audio_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)

    # Compute features
    pitch = np.nanmean(librosa.core.piptrack(y=y, sr=sr)[0])
    tempo = np.mean(librosa.beat.tempo(y=y, sr=sr))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=8), axis=1)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    rms = np.mean(librosa.feature.rms(y=y))

    # Return features as a numpy array
    features = np.array([
        pitch,
        tempo,
        zero_crossing_rate,
        *mfccs,  # Unpacking MFCCs (8 values)
        spectral_centroid,
        spectral_bandwidth,
        chroma_stft,
        rms
    ])

    return features

# Step 2: Load your dataset and extract features
# Assuming you have a CSV file with audio file paths and labels
data = pd.read_csv('')  # Update with your CSV file path

# Initialize lists to hold features and labels
features_list = []
labels = []

# Iterate through the dataset to extract features and corresponding labels
for index, row in data.iterrows():
    audio_file = row['audio_file']  # Column name containing file paths
    label = row['label']  # Column name containing labels

    # Extract features
    features = extract_audio_features(audio_file)
    features_list.append(features)
    labels.append(label)

# Convert to numpy arrays
X = np.array(features_list)
y = np.array(labels)

# Check the shape of features and labels
print("Shape of feature array:", X.shape)
print("Number of labels:", len(y))

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 6: Save the model (optional)
import joblib
joblib.dump(model, 'random_forest_model1.joblib')


FileNotFoundError: [Errno 2] No such file or directory: ''

In [47]:
extract_audio_features(r"C:\Users\mohan\Downloads\Speaker26_000.wav\Speaker26_000.wav")
                       

AttributeError: No librosa.feature attribute pitch